<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/hERG-Retrieve-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 912.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 3.1 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [3]:
target = new_client.target
target_query = target.search('hERG')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q12809', 'xref_name': None, 'xre...",Homo sapiens,HERG,19.0,False,CHEMBL240,"[{'accession': 'Q12809', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Melanin-concentrating hormone receptor 2/HERG,12.0,False,CHEMBL4106188,"[{'accession': 'Q12809', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Homo sapiens,Voltage-gated potassium channel,3.0,False,CHEMBL2362996,"[{'accession': 'P51787', 'component_descriptio...",PROTEIN FAMILY,9606


In [4]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL240'

In [5]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [6]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,305156,[],CHEMBL841079,Inhibition of hERG currents Kv11.1,T,None,None,BAO_0000190,BAO_0000019,...,Homo sapiens,HERG,9606,None,None,IC50,nM,UO_0000065,None,14.0
1,None,305157,[],CHEMBL841078,Inhibitory concentration against hERG currents...,F,None,None,BAO_0000190,BAO_0000019,...,Homo sapiens,HERG,9606,None,None,IC50,nM,UO_0000065,None,3.0
2,None,305244,[],CHEMBL691014,K+ channel blocking activity in human embryoni...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,HERG,9606,None,None,IC50,nM,UO_0000065,None,32.2
3,None,305245,[],CHEMBL691013,K+ channel blocking activity in human embryoni...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,HERG,9606,None,None,IC50,nM,UO_0000065,None,5950.0
4,None,306561,[],CHEMBL691014,K+ channel blocking activity in human embryoni...,F,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,HERG,9606,None,None,IC50,nM,UO_0000065,None,143.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14650,None,24507107,[],CHEMBL5054711,Inhibition of human ERG by manual patch clamp ...,B,None,None,BAO_0000190,BAO_0000019,...,Homo sapiens,HERG,9606,None,None,IC50,uM,UO_0000065,None,10.0
14651,None,24511208,[],CHEMBL5055966,Inhibition of human ERG by patch clamp method,B,None,None,BAO_0000190,BAO_0000019,...,Homo sapiens,HERG,9606,None,None,IC50,uM,UO_0000065,None,30.0
14652,None,24518507,[],CHEMBL5058034,Inhibition of human ERG,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,HERG,9606,None,None,IC50,uM,UO_0000065,None,30.0
14653,None,24518508,[],CHEMBL5058034,Inhibition of human ERG,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,HERG,9606,None,None,IC50,uM,UO_0000065,None,30.0


In [7]:

# considering nM unit for MIC
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
len(df)

9136

In [ ]:
# considering nM unit for MIC
df = df[(df['standard_units'] == '%')]
df = df[(df['relation'] == '=')]
len(df)

In [8]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 20000:
        STATUS.append("Yes") #active
        
    #elif 50000 <= i <= 10000:
        #STATUS.append("No") #inactive
        
    else:
        STATUS.append("No") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

<ipython-input-8-c075fa7cf728>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,305156,[],CHEMBL841079,Inhibition of hERG currents Kv11.1,T,None,None,BAO_0000190,BAO_0000019,...,HERG,9606,None,None,IC50,nM,UO_0000065,None,14.0,No
2,None,305244,[],CHEMBL691014,K+ channel blocking activity in human embryoni...,F,None,None,BAO_0000190,BAO_0000219,...,HERG,9606,None,None,IC50,nM,UO_0000065,None,32.2,No
3,None,305245,[],CHEMBL691013,K+ channel blocking activity in human embryoni...,F,None,None,BAO_0000190,BAO_0000219,...,HERG,9606,None,None,IC50,nM,UO_0000065,None,5950.0,No


In [ ]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i > 80:
        STATUS.append("Yes") #active
        
    elif i == 0:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df.head(3)

In [ ]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

305

In [9]:

df.to_csv('hERG_IC50_nM.csv', index=False)
     